In [ ]:
# Step 0: Setup
!apt-get update -y
!apt-get install -y git git-lfs build-essential cmake python3-pip
!git lfs install

# Step 1: Clone & build llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!cmake -B build
!cmake --build build -j 1

# Step 2: Download Qwen2.5-Coder-3B-Instruct
%cd /content
!git clone https://huggingface.co/Qwen/Qwen2.5-Coder-3B-Instruct

# Step 3: Convert HF → GGUF (F16)
%cd /content/llama.cpp
!python3 convert_hf_to_gguf.py /content/Qwen2.5-Coder-3B-Instruct \
  --outfile /content/qwen2.5-coder-3b-instruct-f16.gguf

# Step 4: Quantize GGUF → Q4_K_M
!./build/bin/llama-quantize \
  /content/qwen2.5-coder-3b-instruct-f16.gguf \
  /content/qwen2.5-coder-3b-instruct-q4_k_m.gguf \
  Q4_K_M

# Step 5: Check size
!ls -lh /content/*.gguf

# Step 6: Download
# from google.colab import files
# files.download("/content/qwen2.5-coder-3b-instruct-q4_k_m.gguf")

# Donwload from the /content

print("This is the end")

In [2]:
# Full Reset
!rm -f /content/qwen2.5-coder-3b-instruct-q4_k_m.gguf
# !rm -rf /content

In [ ]:
# Verify the f16 file
# It should be around 6.2G
!ls -lh /content/qwen2.5-coder-3b-instruct-f16.gguf

# Verify the quantized model
# This should be like ~2GB
!ls -lh /content/qwen2.5-coder-3b-instruct-q4_k_m.gguf


In [ ]:
# Testing
! /content/llama.cpp/build/bin/llama-cli \
  -m /content/qwen2.5-coder-3b-instruct-q4_k_m.gguf \
  -p "Write python code to print hello world" \
  -n 40
